In [6]:
import riva.client

In [7]:
import soundfile as sf

In [8]:
import grpc
import io
import IPython.display as ipd
import wave

In [9]:
auth = riva.client.Auth(uri='localhost:50051')
riva_asr = riva.client.ASRService(auth)

In [10]:
AUDIO_FILE = 'harvard.wav'
with wave.open(AUDIO_FILE, "rb") as wav:
    audio = wav.readframes(wav.getnframes())
# ipd.Audio(audio, rate=16000)

In [11]:
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",  
    max_alternatives=1,
    enable_automatic_punctuation=True,
    audio_channel_count = 1   
)

In [12]:
response = riva_asr.offline_recognize(audio, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript:", asr_best_transcript)

print("\n\nFull Response Message:")
print(response)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Error: Unavailable model requested given these parameters: language_code=vi; sample_rate=16000; type=offline; "
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:50051 {created_time:"2025-11-12T19:16:38.3790272+00:00", grpc_status:3, grpc_message:"Error: Unavailable model requested given these parameters: language_code=vi; sample_rate=16000; type=offline; "}"
>

In [ ]:
import riva.client
from riva.client import ASRService
import wave

# Địa chỉ server
RIVA_SERVER = "localhost:50051"

# Kết nối Riva
auth = riva.client.Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# File âm thanh test (16kHz, mono, wav)
AUDIO_FILE = "harvard.wav"

with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Cấu hình ASR
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)
print("🗣️  Kết quả nhận dạng:")
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]

# Gọi inference
responses = riva_asr.streaming_response_generator(
    audio_chunks(audio_bytes),
    config,
    end_of_stream=True  # ⚠️ báo audio đã hết
)
print("🗣️  Kết quả nhận dạng:")
for response in responses:
    for result in response.results:
        if len(result.alternatives) > 0:
            print("Transcript:", result.alternatives[0].transcript)

🗣️  Kết quả nhận dạng:


TypeError: ASRService.streaming_response_generator() got an unexpected keyword argument 'end_of_stream'

In [ ]:
import wave
import riva.client
from riva.client import ASRService

# Kết nối server
RIVA_SERVER = "localhost:50051"
auth = riva.client.Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# Đọc file WAV 16kHz mono
AUDIO_FILE = "harvard.wav"
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Generator chia chunk và báo end-of-stream
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]
    yield b""  # ⚠️ kết thúc streaming

# Cấu hình ASR
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

# Gọi streaming inference
responses = riva_asr.streaming_response_generator(audio_chunks(audio_bytes), config)

print("🗣️  Kết quả nhận dạng:")



🗣️  Kết quả nhận dạng:


In [ ]:
responses

<generator object ASRService.streaming_response_generator at 0x0000022A94CD2110>

In [16]:
import wave
from riva.client import ASRService, RecognitionConfig, AudioEncoding, Auth

auth = Auth(uri="localhost:50051")
riva_asr = ASRService(auth)

AUDIO_FILE = "output_16k.wav"
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]
    yield b""  # báo kết thúc stream

config = RecognitionConfig(
    encoding=AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

responses = riva_asr.streaming_response_generator(audio_chunks(audio_bytes), config)

for response in responses:
    for result in response.results:
        for alt in result.alternatives:
            transcript = alt.transcript
            speaker = getattr(alt, "speaker_tag", None)
            if speaker:
                print(f"[Speaker {speaker}]: {transcript}")
            else:
                print(f"[Transcript]: {transcript}")


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception iterating requests!"
	debug_error_string = "None"
>

In [3]:
import librosa
import soundfile as sf

# Đọc file gốc
y, sr = librosa.load("harvard.wav", sr=None)  # sr=None để giữ nguyên sample rate gốc

# Resample sang 16kHz
y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

# Ghi lại
sf.write("output_16k.wav", y_16k, 16000)


d:\Anaconda\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
d:\Anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
d:\Anaconda\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [17]:
import wave
from riva.client import ASRService, RecognitionConfig, AudioEncoding, Auth

# =========================
# 1️⃣ Kết nối Riva Server
# =========================
RIVA_SERVER = "localhost:50051"
auth = Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# =========================
# 2️⃣ File audio multi-speaker
# =========================
AUDIO_FILE = "multi_speaker.wav"  # WAV 16kHz mono, nhiều giọng
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# =========================
# 3️⃣ Generator chia chunk và báo end-of-stream
# =========================
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]
    yield b""  # ⚠️ báo audio kết thúc

# =========================
# 4️⃣ Cấu hình ASR + Diarizer
# =========================
config = RecognitionConfig(
    encoding=AudioEncoding.LINEAR_PCM,
    sample_rate_hz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

# =========================
# 5️⃣ Streaming inference
# =========================
responses = riva_asr.streaming_response_generator(
    audio_chunks(audio_bytes),
    config
)

# =========================
# 6️⃣ Hiển thị transcript có tách giọng
# =========================
print("🗣️ Kết quả nhận dạng với tách giọng:")
for response in responses:
    for result in response.results:
        for alt in result.alternatives:
            transcript = alt.transcript
            speaker = getattr(alt, "speaker_tag", None)
            if speaker is not None:
                print(f"[Speaker {speaker}]: {transcript}")
            else:
                print(f"[Transcript]: {transcript}")


FileNotFoundError: [Errno 2] No such file or directory: 'multi_speaker.wav'

In [18]:
import wave
with wave.open("output_16k.wav", "rb") as wav:
    print(wav.getnchannels(), wav.getsampwidth(), wav.getframerate())

1 2 16000


In [ ]:
import wave
from riva.client import ASRService, RecognitionConfig, AudioEncoding, Auth

# Kết nối server
RIVA_SERVER = "localhost:50051"
auth = Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# Đọc audio 16kHz, mono, PCM16
AUDIO_FILE = "output_16k.wav"
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Cấu hình ASR
config = RecognitionConfig(
    encoding=AudioEncoding.LINEAR_PCM,
    sample_rate_hz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

# Gọi model offline
response = riva_asr.offline_recognize(audio_bytes, config)

# In kết quả
for result in response.results:
    for alt in result.alternatives:
        transcript = alt.transcript
        speaker = getattr(alt, "speaker_tag", None)
        if speaker is not None:
            print(f"[Speaker {speaker}]: {transcript}")
        else:
            print(f"[Transcript]: {transcript}")
